In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

In [2]:
# Set random seed for purposes of reproducibility
seed = 21

In [4]:
from keras.datasets import cifar10

In [5]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 733s 4us/step


In [6]:
# normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [7]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

In [8]:
model = Sequential()

In [9]:
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))

In [10]:
model.add(Dropout(0.2))

In [11]:
model.add(BatchNormalization())

In [12]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

In [13]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [14]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [15]:
model.add(Flatten())
model.add(Dropout(0.2))

In [16]:
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [17]:
model.add(Dense(class_num))
model.add(Activation('softmax'))

In [18]:
epochs = 25
optimizer = 'adam'

In [19]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [20]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0

In [21]:
numpy.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

Epoch 1/25
782/782 [==============================] - 295s 377ms/step - loss: 1.5061 - accuracy: 0.4645 - val_loss: 1.4208 - val_accuracy: 0.5047
Epoch 2/25
782/782 [==============================] - 315s 402ms/step - loss: 1.0252 - accuracy: 0.6368 - val_loss: 0.8876 - val_accuracy: 0.6886
Epoch 3/25
782/782 [==============================] - 295s 378ms/step - loss: 0.8523 - accuracy: 0.6997 - val_loss: 0.7786 - val_accuracy: 0.7239
Epoch 4/25
782/782 [==============================] - 297s 379ms/step - loss: 0.7553 - accuracy: 0.7367 - val_loss: 0.7787 - val_accuracy: 0.7282
Epoch 5/25
782/782 [==============================] - 300s 384ms/step - loss: 0.6994 - accuracy: 0.7559 - val_loss: 0.7079 - val_accuracy: 0.7552
Epoch 6/25
782/782 [==============================] - 303s 387ms/step - loss: 0.6626 - accuracy: 0.7683 - val_loss: 0.6939 - val_accuracy: 0.7538
Epoch 7/25
782/782 [==============================] - 328s 420ms/step - loss: 0.6304 - accuracy: 0.7800 - val_loss: 0.6954 -

In [24]:
# Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 82.51%


In [25]:
model.save('object_recognition.h5')

In [27]:
from keras.models import load_model

In [30]:
load_model('object_recognition.h5')